# Collective Loan in Marlowe


**Executive Summary**

This collective-loan contract lets lenders deposit currency such as `DjedUSD` in exchange for liquidity tokens, and then lets borrowers draw funds from the contract as loans. After the borrowers pay back their borrowings with interest to the collective loan, the lenders then redeem their liquidity tokens for `DjedUSD` that includes profit from the lending. A central administrator manages approvals. The Marlowe contract for this collective loan is scalable to an arbitrarily largenumber of lenders or borrowers, so long as the trusted central administrator is relied upon to disperse role tokens on a just-in-time basis. (Less centralized versions may be feasible, but they would be more rigid in terms of sequence of participation of the lenders and borrowers.) The contract was both simulated and run on the Cardano blockchain: that involved 3 lenders, 5 borrowers, and ~60 transactions.

[A simple one-lender, one-borrower version of this contract](collective-loan.blockly) is available for use in [Marlowe Playground's Blockly/Marlowe format](https://marlowe-playground-staging.plutus.aws.iohkdev.io/#/marlowe).

See also [this video](https://vimeo.com/735889853/10601168e4) that walks through this example in Marlowe Playground and on the Marlowe testnet.


**Highlights**

*   Arbitrary scalability for Marlowe contracts where role-tokens are dispersed on a just-in-time basis.
*   Incremental merkleization for rapid construction of large Marlowe contracts.
*   Novel merkleization of timeouts to limit contract depth.


***Life cycle of this collective loan:***

1.  The administrator configures and creates the contract.
2.  The administrator deposits the liquidity tokens for the contract.
3.  Lenders request to deposit their investment funds.
    1.  The administrator approves the lender.
    2.  The administrator sends a role token to the lender.
    3.  The lender deposits their funds.
    4.  The lender receives liquidity tokens as a receipt.
4.  Borrowers requests loans from the collective loan.
    1.  The administrator authorizes the loan.
    2.  The administrator sends a role token to the borrower.
    3.  The borrower withdraws their funds.
5.  Borrowers deposit their loan proceeds.
6.  Lenders redeem their liquidity tokens.
7.  The administrator closes the contract.


***Caveats and limitations***

-   The contract has a finite life cycle with phases of investment, lending, repayment, and redemption.
-   The administrator may be an organization or an oracle, manual or automated, but it must be highly secure.
    -   For example, a contract with 100 lenders and 5000 loans will fit-on chain if it is merkleized: such a contract contains 20,601 small continuations that account for all possible logical paths through the contract.
    -   However, we have not studied how the arbitrary *interleaving* of lending and payback may affect scalability.
-   Unlike fully decentralized Marlowe contracts, this collective-loan contract relies upon centralized management of role tokens to bypass the Marlowe's limitations on the number of state variables (which are stored in Plutus `Datum`).
    -   The use of reference scripts (CIP-33) in the Babbage Era may somewhat alleviate this limitation.
    -   Role tokens can be pre-distributed *if lenders and borrowers submit transactions in a pre-defined order*: this will decrease centralization and increase security at the expense of making the contract more rigid.

## Sequence Diagram

![UML sequence diagram for collective loan](collective-loan.png)

## Preliminaries

### Versions

Make a note of the versions of the Marlowe repository and software.

In [1]:
marlowe-cli --version

marlowe-cli 0.0.5.0


In [2]:
cardano-cli --version

cardano-cli 1.34.1 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [3]:
git rev-parse HEAD

1e0695fb623d353d3a5575f9977b21faaa53df6c


### Utility Functions

The following functions simplify UTxO management.

In [4]:
function find_role_token() {
  marlowe-cli util select --asset-only $ROLES_POLICY.$2 $1 | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/'
}

In [5]:
function find_ada() {
  marlowe-cli util select --lovelace-only $((10 * ADA)) $1 | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/' | head -n 1
}

In [6]:
function find_token() {
  marlowe-cli util select --asset-only $2 $1 | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/' | head -n 1
}

In [7]:
function find_token_2() {
  marlowe-cli util select --asset-only $2 $1 | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/' | tail -n +2 | head -n 1
}

In [8]:
function make_role_txout () {
  echo "$1+$MIN_ADA+1 $ROLES_POLICY.$2"
}

In [9]:
function make_token_txout() {
  echo "$1+$MIN_ADA+$3 $2"
}

### Select the Cardano network

In [10]:
export CARDANO_TESTNET_MAGIC=1567
export CARDANO_NODE_SOCKET_PATH=node.socket

### Current Time

We'll start the contract at the current time.

In [11]:
NOW=$(($(date -u +%s) * 1000))
echo $NOW

1659630034000


Compute time constants, for convenience when setting time intervals for transactions.

In [12]:
MINUTE=$((60 * 1000))
HOUR=$((60 * MINUTE))
DAY=$((24 * HOUR))

There is a minimum Ada requirement associated with a native token.

In [13]:
ADA=1000000
MIN_ADA=$((2 * ADA))
echo $MIN_ADA

2000000


### Contract Parameters

Set parameters for the contract. This contract scales to large numbers of lenders and borrowers, but that will involve many transactions and much time if run on-chain.

In [14]:
N_LENDERS=3
N_BORROWERS=5

MINIMUM_INVESTMENT=500000
MAXIMUM_INVESTMENT=5000000

POOL_SIZE=$((N_LENDERS * MAXIMUM_INVESTMENT))

MINIMUM_LOAN=100000
MAXIMUM_LOAN=1000000
BORROWER_DJED=500000

TOTAL_DJED=$((POOL_SIZE + N_BORROWERS * BORROWER_DJED))

SETUP_DEADLINE=$((NOW + 1 * HOUR))

INVESTMENT_DEADLINE=$((NOW + 3 * HOUR))
INVESTMENT_REPAYMENT_DEADLINE=$((NOW + 12 * HOUR))

LOAN_DEADLINE=$((NOW + 6 * HOUR))
LOAN_REPAYMENT_DEADLINE=$((NOW + 9 * HOUR))

ULTIMATE_DEADLINE=$((NOW + 15 * HOUR))

### Keys for the Administrator

Use a pre-existing key for payment.

In [15]:
ADMIN_ROLE=A
ADMIN_SKEY=christopher-marlowe.skey
ADMIN_VKEY=christopher-marlowe.vkey
ADMIN_ADDR=$(cat christopher-marlowe.testnet.address)
echo $ADMIN_ADDR

addr_test1vrssw4edcts00kk6lp7p5n64666m23tpprqaarmdwkaq69gfvqnpz


Send 1000 Ada to the administrator.

In [16]:
marlowe-cli util faucet --lovelace $((1000 * ADA)) --out-file /dev/null --submit 600 $ADMIN_ADDR

TxId "c989e98a7f5917d29306bb4b86f31a23358a994501839ca4f52ae7c3b687c87a"


### Key Derivation

We'll derive all of the keys for lenders and borrowers from a single root private key.

In [17]:
ROOT_PRV=william-shakespeare.root.prv

For the convenience of viewing transactions in Daedalus, we'll include the same stake credential in all of the addresses.

In [18]:
cardano-wallet key child 1852H/1815H/0H/2/0 < $ROOT_PRV \
| cardano-cli key convert-cardano-address-key --shelley-stake-key --signing-key-file /dev/stdin --out-file /dev/stdout \
| cardano-cli key verification-key --signing-key-file /dev/stdin --verification-key-file /dev/stdout \
| cardano-cli key non-extended-key --extended-verification-key-file /dev/stdin --verification-key-file william-shakespeare.stake.vkey
cardano-cli stake-address build --testnet-magic $CARDANO_TESTNET_MAGIC --stake-verification-key-file william-shakespeare.stake.vkey

stake_test1urplvp2a7dythh6yxutd0qlzzkncr3gy5ftvxj02d3etafqjugc5h


### Keys for Lenders

Build the signing keys and payment addresses for the lenders from the `1852H/1815H/0H/0/` sequence of key derivations.

In [19]:
mkdir -p tmp
for j in `seq 1 $N_LENDERS`
do
  LENDER_SKEY[$j]=tmp/lender-$j.skey
  LENDER_ADDR[$j]=$(
  cardano-wallet key child 1852H/1815H/0H/0/$j < $ROOT_PRV \
  | cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file /dev/stdin --out-file ${LENDER_SKEY[$j]}
  cardano-cli key verification-key --signing-key-file ${LENDER_SKEY[$j]} --verification-key-file /dev/stdout \
  | cardano-cli address build --testnet-magic $CARDANO_TESTNET_MAGIC --payment-verification-key-file /dev/stdin --stake-verification-key-file william-shakespeare.stake.vkey \
  )
  echo "Lender 1852H/1815H/0H/0/$j = ${LENDER_ADDR[$j]}"
done

Lender 1852H/1815H/0H/0/1 = addr_test1qzenwj7elwatk3mmc368mwnv067fqnuk3x7u4nqw5dezg8wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqf6k36p
Lender 1852H/1815H/0H/0/2 = addr_test1qraxynufcduk7ak7nke2jm6pnc32vn7wsmm7ml30cutfgg7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq77wfgp
Lender 1852H/1815H/0H/0/3 = addr_test1qzlwepx5u26a8w94kfrkfem6njlam22emyhjscu0lxqrce7r7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq5jnrn3


Send 25 ADA to each lender.

In [20]:
marlowe-cli util faucet --lovelace $((25 * ADA)) --out-file /dev/null --submit 600 ${LENDER_ADDR[@]}

TxId "9f0da9216494d506a14f7cc5f48644f13ff3a01a14cc8aa731fb6ae0dcaba588"


### Keys for Borrowers

Build the signing keys and payment addresses for the borrowers from the `1852H/1815H/0H/1/` sequence of key derivations.

In [21]:
mkdir -p tmp
for j in `seq 1 $N_BORROWERS`
do
  BORROWER_SKEY[$j]=tmp/borrower-$j.skey
  BORROWER_ADDR[$j]=$(
    cardano-wallet key child 1852H/1815H/0H/1/$j < $ROOT_PRV \
  | cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file /dev/stdin --out-file ${BORROWER_SKEY[$j]}
  cardano-cli key verification-key --signing-key-file ${BORROWER_SKEY[$j]} --verification-key-file /dev/stdout \
  | cardano-cli address build --testnet-magic $CARDANO_TESTNET_MAGIC --payment-verification-key-file /dev/stdin --stake-verification-key-file william-shakespeare.stake.vkey \
  )
  echo "Borrower 1852H/1815H/0H/1/$j = ${BORROWER_ADDR[$j]}"
done

Borrower 1852H/1815H/0H/1/1 = addr_test1qp7q4e9nlhq08mrugs8pm5uq045e425w72mxhxupq9qqgykr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqjmt3lh
Borrower 1852H/1815H/0H/1/2 = addr_test1qpzun0s6thsyf9qt0zyfnn50x8kpwutrv20eq8dgx9gt6tkr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqfsjdsz
Borrower 1852H/1815H/0H/1/3 = addr_test1qqpet7s339suufnxmr4vrg09469v30g59yxa70yrkxvrfekr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq37csh4
Borrower 1852H/1815H/0H/1/4 = addr_test1qppr8ue92zznn2k3v235fuxtpcun9k32g8jn09dsq3ndvawr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqh2lydz
Borrower 1852H/1815H/0H/1/5 = addr_test1qzcee8xm57tuss5wm3yvxhpk2w7ws3h03c8w2f30tlqahhkr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqgtpxaq


Send 25 ADA to each borrower.

In [22]:
marlowe-cli util faucet --lovelace $((25 * ADA)) --out-file /dev/null --submit 600 ${BORROWER_ADDR[@]}

TxId "1e6419b586e4e3c9e73855c51dd6f6dedde5816ffe0d062f3ce11ded80a1c394"


### Mint the Djed

The contract will use fake Djed for payments.

In [23]:
DJED_NAME=DjedUSD

In [24]:
DJED_POLICY=$(
marlowe-cli util mint --required-signer $ADMIN_SKEY \
                      --change-address $ADMIN_ADDR  \
                      --expires 1000000011          \
                      --count $TOTAL_DJED           \
                      --out-file /dev/null          \
                      --submit 600                  \
                      $DJED_NAME                    \
| sed -e 's/^PolicyID "\(.*\)"$/\1/'                \
)
echo 'PolicyID "'$DJED_POLICY'"'

PolicyID "76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf"


In [25]:
DJED=$DJED_POLICY.$DJED_NAME
echo $DJED

76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.DjedUSD


In `cardano-cli` the token name is printed in hexadecimal, so we have . . .

In [26]:
echo $DJED_POLICY.$(echo -n $DJED_NAME | basenc --base16 | tr '[:upper:]' '[:lower:]')

76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344


### Distribute the Djed.

Send Djed to lenders and borrowers.

In [27]:
TXOUT=()
for j in `seq 1 $N_LENDERS`
do
  TXOUT+=("--tx-out" "${LENDER_ADDR[$j]}+$MIN_ADA+$MAXIMUM_INVESTMENT\ $DJED")
done
for j in `seq 1 $N_BORROWERS`
do
  TXOUT+=("--tx-out" "${BORROWER_ADDR[$j]}+$MIN_ADA+$BORROWER_DJED\ $DJED")
done

In [28]:
echo marlowe-cli transaction simple --tx-in "$(find_ada $ADMIN_ADDR)"         \
                                    --tx-in "$(find_token $ADMIN_ADDR $DJED)" \
                                    ${TXOUT[@]}                               \
                                    --change-address $ADMIN_ADDR              \
                                    --required-signer $ADMIN_SKEY             \
                                    --out-file /dev/null                      \
                                    --submit 600                              \
| bash

TxId "2de3b3cd9513312eed16b01897d20f3bc7609978de10253e432825a8b76b4b2c"


### Mint the Liquidity Tokens

The contract will use liquidity tokens for bookkeeping.

In [29]:
LIQUIDITY_NAME=Liquidity

In [30]:
LIQUIDITY_POLICY=$(
marlowe-cli util mint --required-signer $ADMIN_SKEY \
                      --change-address $ADMIN_ADDR  \
                      --expires 1000000012          \
                      --count $POOL_SIZE            \
                      --out-file /dev/null          \
                      --submit 600                  \
                      $LIQUIDITY_NAME               \
| sed -e 's/^PolicyID "\(.*\)"$/\1/'                \
)
echo 'PolicyID "'$LIQUIDITY_POLICY'"'

PolicyID "5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd"


In [31]:
LIQUIDITY=$LIQUIDITY_POLICY.$LIQUIDITY_NAME
echo $LIQUIDITY

5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.Liquidity


In `cardano-cli` the token name is printed in hexadecimal, so we have . . .

In [32]:
echo $LIQUIDITY_POLICY.$(echo -n $LIQUIDITY_NAME | basenc --base16 | tr '[:upper:]' '[:lower:]')

5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479


### Mint the Role Tokens

There is one role token for the administrator.

In [33]:
ADMIN_ROLE=A

Lender and borrower tokens will be distribute on a just-in-time basis for making deposits and (separately) for making withdrawals. Note that ordering of these is arbitrary and does not necessarily correspond to the lender and borrower addresses.

In [34]:
for j in `seq 1 $N_LENDERS`
do
  LENDER_DEPOSIT_ROLE[$j]=LD$j
  LENDER_PAYMENT_ROLE[$j]=LP$j
done

In [35]:
for j in `seq 1 $N_BORROWERS`
do
  BORROWER_DEPOSIT_ROLE[$j]=BD$j
  BORROWER_PAYMENT_ROLE[$j]=BP$j
done

Mint the role tokens.

In [36]:
ROLES_POLICY=$(
marlowe-cli util mint --required-signer $ADMIN_SKEY \
                      --change-address $ADMIN_ADDR  \
                      --expires 1000000013          \
                      --out-file /dev/null          \
                      --submit 600                  \
                      $ADMIN_ROLE                   \
                      ${LENDER_DEPOSIT_ROLE[@]}     \
                      ${LENDER_PAYMENT_ROLE[@]}     \
                      ${BORROWER_DEPOSIT_ROLE[@]}   \
                      ${BORROWER_PAYMENT_ROLE[@]}   \
| sed -e 's/^PolicyID "\(.*\)"$/\1/'                \
)
echo 'PolicyID "'$ROLES_POLICY'"'

PolicyID "9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064e78"


View the role tokens.

In [37]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ADMIN_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
c36871383d1f8045eaf35adecc87c60976e164f2835b343d212255e8151464e8     0        1852833722 lovelace + TxOutDatumNone
c36871383d1f8045eaf35adecc87c60976e164f2835b343d212255e8151464e8     1        10000000 lovelace + 15000000 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + TxOutDatumNone
c36871383d1f8045eaf35adecc87c60976e164f2835b343d212255e8151464e8     2        10000000 lovelace + 1 9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064e78.41 + TxOutDatumNone
c36871383d1f8045eaf35adecc87c60976e164f2835b343d212255e8151464e8     3        10000000 lovelace + 1 9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064e78.424431 + TxOutDatumNone
c36871383d1f8045eaf35adecc87c60976e164f2835b343d212255e8151464e8     4        10000000 lovelace + 1 9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064e7

## Generate the Contract

Run a Haskell program that generates the contract.

In [38]:
runhaskell << EOI

{-# LANGUAGE OverloadedStrings  #-}


module CollectiveLoan where


import Control.Monad.Writer (Writer, runWriter, tell)
import Data.Aeson ((.=), encodeFile,object)
import Data.Foldable (foldrM)
import Data.String (fromString)
import Language.Marlowe.CLI.Merkle (deepMerkleize)
import Language.Marlowe.CLI.Types (Continuations)
import Language.Marlowe.Core.V1.Semantics.Types
import Plutus.V1.Ledger.Api (POSIXTime(..))


main :: IO ()
main =
  let
    (contract, continuations) =
      runWriter
        $ makeContract
          $POOL_SIZE $SETUP_DEADLINE
          $N_LENDERS   $MINIMUM_INVESTMENT $MAXIMUM_INVESTMENT $INVESTMENT_DEADLINE     $INVESTMENT_REPAYMENT_DEADLINE
          $N_BORROWERS $MINIMUM_LOAN       $MAXIMUM_LOAN       $LOAN_REPAYMENT_DEADLINE $LOAN_DEADLINE
  in
    encodeFile "collective-loan.merkleization"
      $ object
        [
          "contract"      .= contract
        , "continuations" .= continuations
        ]


djed :: Token
djed = Token "$DJED_POLICY" "$DJED_NAME"

liquidity :: Token
liquidity = Token "$LIQUIDITY_POLICY" "$LIQUIDITY_NAME"

equity :: ValueId
equity = "Equity"

liability :: ValueId
liability = "Liability"

administrator :: Party
administrator = Role "$ADMIN_ROLE"


merkleizeTimeout :: Contract -> Writer Continuations Contract
merkleizeTimeout continuation =
  deepMerkleize
    $ When
      [
        Case (Notify TrueObs )
        continuation
      ]
      (POSIXTime $ULTIMATE_DEADLINE)
      Close


makeContract :: Integer -> Timeout
             -> Int -> Integer -> Integer -> Timeout -> Timeout
             -> Int -> Integer -> Integer -> Timeout -> Timeout
             -> Writer Continuations Contract
makeContract poolSize setupDeadline
         nLenders minimumInvestment maximumInvestment lenderDepositDeadline lenderWithdrawalDeadline
         nBorrowers   minimumLoan       maximumLoan       borrowerDepositDeadline   borrowerWithdrawalDeadline =
  let
    lenders = [1..nLenders]
    borrowers = [1..nBorrowers]
    z c = flip (foldrM (lenderWithdraws maximumInvestment lenderWithdrawalDeadline)) lenders c
  in
    setup poolSize setupDeadline
      =<< flip (foldrM $ lenderDeposits minimumInvestment maximumInvestment lenderDepositDeadline) lenders
      =<< pure . computeLiability
      =<< flip (foldrM $ borrowerWithdraws minimumLoan maximumLoan borrowerWithdrawalDeadline) borrowers
      =<< flip (foldrM $ borrowerDeposits maximumLoan borrowerDepositDeadline) borrowers
      =<< pure . computeEquity
      =<< flip (foldrM $ lenderWithdraws maximumInvestment lenderWithdrawalDeadline) lenders
      =<< merkleizeTimeout Close


setup :: Integer -> Timeout -> Contract -> Writer Continuations Contract
setup poolSize deadline continuation =
  deepMerkleize
    $ When
      [
        Case (Deposit administrator administrator liquidity $ Constant poolSize)
          continuation
      ]
      deadline
      Close


computeLiability :: Contract -> Contract
computeLiability =
  Let "Liability"
    $ AvailableMoney administrator djed


computeEquity :: Contract -> Contract
computeEquity =
  Let "Equity"
    $ AvailableMoney administrator djed


borrowerDeposits :: Integer -> Timeout -> Int -> Contract -> Writer Continuations Contract
borrowerDeposits maximumLoan deadline i continuation =
  let
    name = "Borrower Deposit"
    choice = ChoiceId (fromString name) administrator
    amount = ChoiceValue choice
    borrower = Role . fromString $ "BD" <> show i
  in
    deepMerkleize
      . When
        [
          Case (Choice choice [Bound 1 $ 2 * maximumLoan])
            $ When
               [
                 Case (Deposit administrator borrower djed amount)
                   continuation
               ]
               deadline
               continuation
        ]
        deadline
        =<< merkleizeTimeout continuation


borrowerWithdraws :: Integer -> Integer -> Timeout -> Int -> Contract -> Writer Continuations Contract
borrowerWithdraws minimumLoan maximumLoan deadline i continuation =
  let
    name = "Borrower Withdrawal"
    choice = ChoiceId (fromString name) administrator
    amount = ChoiceValue choice
    borrower = Role . fromString $ "BP" <> show i
  in
    deepMerkleize
      . When
        [
          Case (Choice choice [Bound minimumLoan maximumLoan])
            $ Pay administrator (Party borrower) djed amount
              continuation
        ]
        deadline
        =<< merkleizeTimeout continuation


lenderDeposits :: Integer -> Integer -> Timeout -> Int -> Contract -> Writer Continuations Contract
lenderDeposits minimumInvestment maximumInvestment deadline i continuation =
  let
    name = "Lender Deposit"
    choice = ChoiceId (fromString name) administrator
    amount = ChoiceValue choice
    lender = Role . fromString $ "LD" <> show i
  in
    deepMerkleize
      . When
      [
        Case (Choice choice [Bound minimumInvestment maximumInvestment])
          $ When
              [
                Case (Deposit administrator lender djed amount)
                  $ Pay administrator (Party lender) liquidity amount
                    continuation
              ]
              deadline
              continuation
      ]
      deadline
      =<< merkleizeTimeout continuation


lenderWithdraws :: Integer -> Timeout -> Int -> Contract -> Writer Continuations Contract
lenderWithdraws maximumInvestment deadline i continuation =
  let
    name = "Lender Withdrawal"
    choice = ChoiceId (fromString name) administrator
    amount = ChoiceValue choice
    lender = Role . fromString $ "LP" <> show i
  in
    deepMerkleize
      . When
        [
          Case (Choice choice [Bound 1 $ 2 * maximumInvestment])
            $ When
                [
                  Case (Deposit administrator lender liquidity amount)
                    $ Pay administrator (Party lender) djed
                      (
                        DivValue
                          (MulValue amount $ UseValue equity)
                          (UseValue liability)
                      )
                      continuation
                ]
                deadline
                continuation
        ]
        deadline
        =<< merkleizeTimeout continuation
    
EOI

How large is the contract?

In [39]:
ls -lh collective-loan.merkleization

-rw-rw-r-- 1 bbush bbush-upg 29K Aug  4 10:26 collective-loan.merkleization


How many `Case` statements does it contain?

In [40]:
sed -e 's/case/&\n/g' collective-loan.merkleization | grep case | wc -l

87


How many merkleizations does it contain?

In [41]:
jq '.continuations | length' collective-loan.merkleization

39


Separate the contract from the merkleizations.

In [42]:
jq .contract collective-loan.merkleization > collective-loan-1.contract

## Set the initial state

The Seller will deposit 2 Ada when the contract is created.

In [43]:
yaml2json << EOI > collective-loan-1.state
accounts:
- - - role_token: $ADMIN_ROLE
    - currency_symbol: ''
      token_name: ''
  - $MIN_ADA
boundValues: []
choices: []
minTime: $NOW
EOI

In [44]:
cat collective-loan-1.state

{"accounts":[[[{"role_token":"A"},{"currency_symbol":"","token_name":""}],2000000]],"boundValues":[],"choices":[],"minTime":1659630034000}


## Simulate and Execute the Contract

### 1. Initialize the contract information

Create a `.marlowe` file that contains all of the information needed to start the contract. Note that we merkleization the contract because of its large size.

In [45]:
marlowe-cli run initialize --roles-currency $ROLES_POLICY             \
                           --contract-file collective-loan-1.contract \
                           --state-file    collective-loan-1.state    \
                           --out-file      collective-loan-1.partial  \
                           --print-stats


Validator size: 12611
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 24562825, exBudgetMemory = ExMemory 82600}


Manually merge the continuations into the `.marlowe` file.

In [46]:
jq -s '.[0] * .[1]' collective-loan-1.partial collective-loan.merkleization > collective-loan-1.marlowe

The minimum Ada deposit has been recorded.

In [47]:
jq .state collective-loan-1.marlowe | json2yaml

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
boundValues: []
choices: []
minTime: 1659630034000


Submit the transaction.

In [48]:
TX_1=$(
marlowe-cli run execute --tx-in "$(find_ada $ADMIN_ADDR)"            \
                        --marlowe-out-file collective-loan-1.marlowe \
                        --change-address $ADMIN_ADDR                 \
                        --required-signer $ADMIN_SKEY                \
                        --out-file /dev/null                         \
                        --submit 600                                 \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                     \
)
echo 'TxId "'$TX_1'"'

TxId "8fee1ccb0a2053b61461fdcbee1aaebd2cc53f19684894521cc2eb79298ad720"


View the UTxO at the script address.

In [49]:
SCRIPT_ADDR=$(jq -r .marloweValidator.address collective-loan-1.marlowe)
echo $SCRIPT_ADDR

addr_test1wphttveup3kdrp7ch4e4xjr8jxstwwx20fg8s4epqayylfqga9h6p


In [50]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
8fee1ccb0a2053b61461fdcbee1aaebd2cc53f19684894521cc2eb79298ad720     1        2000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "1e80317bbd90e4b9dd4d84b25fc45c44de2f6440aff3c440c863dd36dae98886"


### 2. The administrator deposits the liquidity tokens

Prepare the transaction.

In [51]:
marlowe-cli run prepare --deposit-account Role=$ADMIN_ROLE                   \
                        --deposit-party Role=$ADMIN_ROLE                     \
                        --deposit-token $LIQUIDITY                           \
                        --deposit-amount $POOL_SIZE                          \
                        --invalid-before $NOW                                \
                        --invalid-hereafter $((SETUP_DEADLINE - 1 * MINUTE)) \
                        --marlowe-file collective-loan-1.marlowe             \
                        --out-file     collective-loan-2.marlowe             \
                        --print-stats


Datum size: 262


Now the contract contains both the initial Ada and the liquidity tokens.

In [52]:
jq .state collective-loan-2.marlowe | json2yaml

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: A
    - currency_symbol: 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd
      token_name: Liquidity
  - 15000000
boundValues: []
choices: []
minTime: 1659630034000


Submit the transaction.

In [53]:
TX_2=$(
marlowe-cli run execute --marlowe-in-file collective-loan-1.marlowe           \
                        --tx-in-marlowe "$TX_1#1"                             \
                        --tx-in-collateral "$TX_1#0"                          \
                        --tx-in "$TX_1#0"                                     \
                        --tx-in "$(find_token $ADMIN_ADDR $LIQUIDITY)"        \
                        --tx-in "$(find_role_token $ADMIN_ADDR $ADMIN_ROLE)"  \
                        --tx-out "$(make_role_txout $ADMIN_ADDR $ADMIN_ROLE)" \
                        --marlowe-out-file collective-loan-2.marlowe          \
                        --change-address $ADMIN_ADDR                          \
                        --required-signer $ADMIN_SKEY                         \
                        --out-file /dev/null                                  \
                        --submit 600                                          \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                              \
)
echo 'TxId "'$TX_2'"'

TxId "2ef8a06cda8f245facbde2b4cf2f3e66350e1d59d089c0287439a934c87c1479"


View the UTxO at the script address.

In [54]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
2ef8a06cda8f245facbde2b4cf2f3e66350e1d59d089c0287439a934c87c1479     1        2000000 lovelace + 15000000 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + TxOutDatumHash ScriptDataInAlonzoEra "aec7b3f4e35ad96d2f148ce073b215d1c974d667430e39590aa74cec455af2a8"


### 3. The lenders deposit their investment funds

Lenders deposit in random order.

In [55]:
INVESTMENT_ORDER=(`seq 1 $N_LENDERS | sort -R`)
echo ${INVESTMENT_ORDER[@]}

1 3 2


Lenders deposit random amounts.

In [56]:
for j in `seq 1 $N_LENDERS`
do
  INVESTMENT_AMOUNT[$j]=$(((MAXIMUM_INVESTMENT - MINIMUM_INVESTMENT) * RANDOM / 32767 + MINIMUM_INVESTMENT))
done
echo ${INVESTMENT_AMOUNT[@]}

671391 2056260 2940824


Make the deposits.

In [57]:
TX_OLD=$TX_2
k=2
for i in `seq 1 $N_LENDERS`
do
  k1=$((k+1))
  k2=$((k+2))
  j=${INVESTMENT_ORDER[$((i-1))]}
  echo
  echo "--- Lender deposit #$i ---"
  ROLE=${LENDER_DEPOSIT_ROLE[$i]}
  ADDR=${LENDER_ADDR[$j]}
  SKEY=${LENDER_SKEY[$j]}
  AMOUNT=${INVESTMENT_AMOUNT[$j]}
  DEADLINE=$((INVESTMENT_DEADLINE - 1 * MINUTE))
  echo "Lender $j requests to deposit $AMOUNT Djed."
  marlowe-cli run prepare --choice-party Role=$ADMIN_ROLE            \
                          --choice-name "Lender Deposit"             \
                          --choice-number $AMOUNT                    \
                          --invalid-before $NOW                      \
                          --invalid-hereafter $DEADLINE              \
                          --marlowe-file collective-loan-$k.marlowe  \
                          --out-file     collective-loan-$k1.marlowe
  echo "Administrator approves deposit and sends role token to lender."
  TX_ADA=$(find_ada $ADMIN_ADDR)
  TX_NEW=$(
  marlowe-cli run execute --marlowe-in-file collective-loan-$k.marlowe          \
                          --tx-in-marlowe "$TX_OLD#1"                           \
                          --tx-in-collateral $TX_ADA                            \
                          --tx-in $TX_ADA                                       \
                          --tx-in "$(find_role_token $ADMIN_ADDR $ADMIN_ROLE)"  \
                          --tx-in "$(find_role_token $ADMIN_ADDR $ROLE)"        \
                          --tx-out "$(make_role_txout $ADMIN_ADDR $ADMIN_ROLE)" \
                          --tx-out "$(make_role_txout $ADDR $ROLE)"             \
                          --marlowe-out-file collective-loan-$k1.marlowe        \
                          --change-address $ADMIN_ADDR                          \
                          --required-signer $ADMIN_SKEY                         \
                          --out-file /dev/null                                  \
                          --submit 600                                          \
  | sed -e 's/^TxId "\(.*\)"$/\1/'                                              \
  )
  echo 'TxId "'$TX_NEW'"'
  TX_OLD=$TX_NEW
  echo "Lender deposits Djed and receives liquidity tokens using their newly-received role token."
  marlowe-cli run prepare --deposit-account Role=$ADMIN_ROLE         \
                          --deposit-party Role=$ROLE                 \
                          --deposit-amount $AMOUNT                   \
                          --deposit-token $DJED                      \
                          --invalid-before $NOW                      \
                          --invalid-hereafter $DEADLINE              \
                          --marlowe-file collective-loan-$k1.marlowe \
                          --out-file     collective-loan-$k2.marlowe 
  TX_ADA=$(find_ada $ADDR)
  CHANGE=$((MAXIMUM_INVESTMENT - AMOUNT))
  TX_NEW=$(
  marlowe-cli run execute --marlowe-in-file collective-loan-$k1.marlowe      \
                          --tx-in-marlowe "$TX_OLD#1"                        \
                          --tx-in-collateral $TX_ADA                         \
                          --tx-in $TX_ADA                                    \
                          --tx-in "$(find_token $ADDR $DJED)"                \
                          --tx-in "$(find_role_token $ADDR $ROLE)"           \
                          --tx-out "$(make_role_txout $ADDR $ROLE)"          \
                          --tx-out "$(make_token_txout $ADDR $DJED $CHANGE)" \
                          --marlowe-out-file collective-loan-$k2.marlowe     \
                          --change-address $ADDR                             \
                          --required-signer $SKEY                            \
                          --out-file /dev/null                               \
                          --submit 600                                       \
  | sed -e 's/^TxId "\(.*\)"$/\1/'                                           \
  )
  echo 'TxId "'$TX_NEW'"'
  TX_OLD=$TX_NEW
  k=$k2
done


--- Lender deposit #1 ---
Lender 1 requests to deposit 671391 Djed.
Administrator approves deposit and sends role token to lender.
TxId "f65343bec00be343395916d078e68e33be6de8990a7f271cf9b0cf381fa88532"
Lender deposits Djed and receives liquidity tokens using their newly-received role token.
Payment 1
  Acccount: "A"
  Payee: Party "LD1"
  Ada: 0.000000
  5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd."Liquidity": 671391
TxId "74a140c663933c63dd249b547f46690d52ee00798ff1c40466e9818d900997eb"

--- Lender deposit #2 ---
Lender 3 requests to deposit 2940824 Djed.
Administrator approves deposit and sends role token to lender.
TxId "b073b18a30409cb85980f1f43c1d327bd4807e91a4f0b0b31f2ba3b25aeb3609"
Lender deposits Djed and receives liquidity tokens using their newly-received role token.
Payment 1
  Acccount: "A"
  Payee: Party "LD2"
  Ada: 0.000000
  5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd."Liquidity": 2940824
TxId "6fdb27bf00390045c20252dec592c876ca591e55c4ed0ff1

View the state of the contract.

In [58]:
jq .state collective-loan-$k.marlowe | json2yaml

accounts:
- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: A
    - currency_symbol: 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd
      token_name: Liquidity
  - 9331525
- - - role_token: A
    - currency_symbol: 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
      token_name: DjedUSD
  - 5668475
boundValues:
- - Liability
  - 5668475
choices:
- - choice_name: Lender Deposit
    choice_owner:
      role_token: A
  - 2056260
minTime: 1659630034000


View the UTxO at the script address.

In [59]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5251fd1d58153bc177f3956a0ac482353aef39a47c68dc533d7dac01e2064646     1        2000000 lovelace + 9331525 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + 5668475 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344 + TxOutDatumHash ScriptDataInAlonzoEra "37079b677f0e6742b4f607ec93a766416c180362805cff555a7a7dcf5bf178eb"


### 4. The lenders withdraw their liquidity tokens

The role-payout validator address contains the liquidity tokens that were paid to the lenders.

In [60]:
ROLES_ADDR=$(jq -r .rolesValidator.address collective-loan-1.marlowe)
echo $ROLES_ADDR

addr_test1wq0fq8ghuemq7cjk4cd3fys20g6ew4ntpv7tpw4fclat8nqs93zsf


View these payouts.

In [61]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ROLES_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
5251fd1d58153bc177f3956a0ac482353aef39a47c68dc533d7dac01e2064646     2        1724100 lovelace + 2056260 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + TxOutDatumHash ScriptDataInAlonzoEra "07ef8ca0dad57b0913defa384212e3d4dcb833185743e73109a0479117a776b2"
6fdb27bf00390045c20252dec592c876ca591e55c4ed0ff1fabd842f2580c688     2        1724100 lovelace + 2940824 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + TxOutDatumHash ScriptDataInAlonzoEra "f5d8260020d67f4c9104d139318b1b1266db527b51ee2baf2a1fe426e5730066"
74a140c663933c63dd249b547f46690d52ee00798ff1c40466e9818d900997eb     2        1724100 lovelace + 671391 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + TxOutDatumHash ScriptDataInAlonzoEra "459dc064a604ac976036dba2db4f1c1a4d

Withdraw these payouts to the lenders' addresses.

In [62]:
for i in `seq 1 $N_LENDERS`
do
  j=${INVESTMENT_ORDER[$((i-1))]}
  ROLE=${LENDER_DEPOSIT_ROLE[$i]}
  ADDR=${LENDER_ADDR[$j]}
  SKEY=${LENDER_SKEY[$j]}
  TX_ADA=$(find_ada $ADDR)
  marlowe-cli run withdraw --marlowe-file collective-loan-$((2*j+2)).marlowe \
                           --role-name $ROLE                                 \
                           --tx-in-collateral $TX_ADA                        \
                           --tx-in $TX_ADA                                   \
                           --tx-in $(find_role_token $ADDR $ROLE)            \
                           --tx-out "$(make_role_txout $ADDR $ROLE)"         \
                           --change-address $ADDR                            \
                           --required-signer $SKEY                           \
                           --out-file /dev/null                              \
                           --submit 600
done

TxId "28d0e4eedeffac85f808c05e60d3583b05921658ca92277a71be8f3cc585cedf"
TxId "bfafcd0c51041f6bc19695358d03370590e7d1b5fc30d716362a40aead01c635"
TxId "2c2f01a39e7a81315184d38bea71f3389c31df0ce4d7dce9e207bd7ba19b2c91"


There are no UTxOs at the payout address.

In [63]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ROLES_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


View the UTxOs at the lenders' addresses to see that they have their role tokens and liquidity tokens.

In [64]:
for j in `seq 1 $N_LENDERS`
do
  echo
  echo "Lender $j = ${LENDER_ADDR[$j]}"
  cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address ${LENDER_ADDR[$j]}
done


Lender 1 = addr_test1qzenwj7elwatk3mmc368mwnv067fqnuk3x7u4nqw5dezg8wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqf6k36p
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
28d0e4eedeffac85f808c05e60d3583b05921658ca92277a71be8f3cc585cedf     0        19470339 lovelace + TxOutDatumNone
28d0e4eedeffac85f808c05e60d3583b05921658ca92277a71be8f3cc585cedf     1        1724100 lovelace + 671391 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + TxOutDatumNone
28d0e4eedeffac85f808c05e60d3583b05921658ca92277a71be8f3cc585cedf     2        2000000 lovelace + 1 9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064e78.4c4431 + TxOutDatumNone
74a140c663933c63dd249b547f46690d52ee00798ff1c40466e9818d900997eb     4        2000000 lovelace + 4328609 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344 + TxOutDatumNone
74a140c663933c63dd249b5

### 5. The borrowers withdraw their loan funds

Borrowers withdraw in random order.

In [65]:
BORROWER_ORDER=(`seq 1 $N_BORROWERS | sort -R`)
echo ${BORROWER_ORDER[@]}

3 5 2 1 4


Borrowers deposit random amounts.

In [66]:
for j in `seq 1 $N_BORROWERS`
do
  LOAN_AMOUNT[$j]=$(((MAXIMUM_LOAN - MINIMUM_LOAN) * RANDOM / 32767 + MINIMUM_LOAN))
done
echo ${LOAN_AMOUNT[@]}

993462 239613 932569 222748 920703


Make the withdrawals.

In [67]:
for i in `seq 1 $N_BORROWERS`
do
  k1=$((k+1))
  j=${BORROWER_ORDER[$((i-1))]}
  echo
  echo "--- Borrower withdrawal #$i ---"
  ROLE=${BORROWER_PAYMENT_ROLE[$i]}
  ADDR=${BORROWER_ADDR[$j]}
  SKEY=${BORROWER_SKEY[$j]}
  AMOUNT=${LOAN_AMOUNT[$j]}
  echo "Borrower $j requests loan of $AMOUNT Djed."
  echo "Administrator checks credit."
  echo "Administator approves loan and sends role token to borrower."
  DEADLINE=$((LOAN_DEADLINE - 1 * MINUTE))
  marlowe-cli run prepare --choice-party Role=$ADMIN_ROLE            \
                          --choice-name "Borrower Withdrawal"        \
                          --choice-number $AMOUNT                    \
                          --invalid-before $NOW                      \
                          --invalid-hereafter $DEADLINE              \
                          --marlowe-file collective-loan-$k.marlowe  \
                          --out-file     collective-loan-$k1.marlowe
  TX_ADA=$(find_ada $ADMIN_ADDR)
  TX_NEW=$(
  marlowe-cli run execute --marlowe-in-file collective-loan-$k.marlowe          \
                          --tx-in-marlowe "$TX_OLD#1"                           \
                          --tx-in-collateral $TX_ADA                            \
                          --tx-in $TX_ADA                                       \
                          --tx-in "$(find_role_token $ADMIN_ADDR $ADMIN_ROLE)"  \
                          --tx-in "$(find_role_token $ADMIN_ADDR $ROLE)"        \
                          --tx-out "$(make_role_txout $ADMIN_ADDR $ADMIN_ROLE)" \
                          --tx-out "$(make_role_txout $ADDR $ROLE)"             \
                          --marlowe-out-file collective-loan-$k1.marlowe        \
                          --change-address $ADMIN_ADDR                          \
                          --required-signer $ADMIN_SKEY                         \
                          --out-file /dev/null                                  \
                          --submit 600                                          \
  | sed -e 's/^TxId "\(.*\)"$/\1/'                                              \
  )
  echo 'TxId "'$TX_NEW'"'
  TX_OLD=$TX_NEW
  echo "Borrower withdraws the Djed from the payout address using their newly-received role token."
  TX_ADA=$(find_ada $ADDR)
  marlowe-cli run withdraw --marlowe-file collective-loan-$k1.marlowe \
                           --role-name $ROLE                          \
                           --tx-in-collateral $TX_ADA                 \
                           --tx-in $TX_ADA                            \
                           --tx-in $(find_role_token $ADDR $ROLE)     \
                           --tx-out "$(make_role_txout $ADDR $ROLE)"  \
                           --change-address $ADDR                     \
                           --required-signer $SKEY                    \
                           --out-file /dev/null                       \
                           --submit 600
  k=$k1
done


--- Borrower withdrawal #1 ---
Borrower 3 requests loan of 932569 Djed.
Administrator checks credit.
Administator approves loan and sends role token to borrower.
Payment 1
  Acccount: "A"
  Payee: Party "BP1"
  Ada: 0.000000
  76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf."DjedUSD": 932569
TxId "124b2f805a012ad0e62e5ec9545864b1c88e664a9e59080c7a931ffd141399a8"
Borrower withdraws the Djed from the payout address using their newly-received role token.
TxId "721840ae3bca11c3ffd2a011ca272ae7d893507bbf2e5cce8c5bc96f97584eec"

--- Borrower withdrawal #2 ---
Borrower 5 requests loan of 920703 Djed.
Administrator checks credit.
Administator approves loan and sends role token to borrower.
Payment 1
  Acccount: "A"
  Payee: Party "BP2"
  Ada: 0.000000
  76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf."DjedUSD": 920703
TxId "a1bcf742efc16bbeacbea223f498167b8e211216d0ad173b926f53c7e4f2d04e"
Borrower withdraws the Djed from the payout address using their newly-received role tok

Now the contract accounts show that some Djed remains.

In [68]:
jq .state.accounts collective-loan-$k.marlowe | json2yaml

- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: A
    - currency_symbol: 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd
      token_name: Liquidity
  - 9331525
- - - role_token: A
    - currency_symbol: 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
      token_name: DjedUSD
  - 2359380


View the UTxO at the script address.

In [69]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
bc32bda414d26811e67e534f419df22a8e08215b16383751d755a5ce8c9ba9ba     1        2000000 lovelace + 9331525 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + 2359380 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344 + TxOutDatumHash ScriptDataInAlonzoEra "65fb1f50faff485b01ea310a8565ec9996dbf84a4daa471839a637cd9605dfec"


There are no UTxOs at the payout address.

In [70]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ROLES_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


View the UTxOs at the borrowers' addresses to see that they have their role tokens and Djed.

In [71]:
for j in `seq 1 $N_BORROWERS`
do
  echo
  echo "Borrower $j = ${BORROWER_ADDR[$j]}"
  cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address ${BORROWER_ADDR[$j]}
done


Borrower 1 = addr_test1qp7q4e9nlhq08mrugs8pm5uq045e425w72mxhxupq9qqgykr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqjmt3lh
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
2de3b3cd9513312eed16b01897d20f3bc7609978de10253e432825a8b76b4b2c     4        2000000 lovelace + 500000 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344 + TxOutDatumNone
63c3881dd2769310b9bf707ea8db00589cb0e066857e37c5a2b711261483001b     0        24541621 lovelace + TxOutDatumNone
63c3881dd2769310b9bf707ea8db00589cb0e066857e37c5a2b711261483001b     1        1689618 lovelace + 993462 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344 + TxOutDatumNone
63c3881dd2769310b9bf707ea8db00589cb0e066857e37c5a2b711261483001b     2        2000000 lovelace + 1 9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064e78.425034 + TxOutDatumNone
f6735c9d1bba0eacd5ad37d81a

### 6. The borrowers pay back their funds with interest

Some time passes, and the borrowers pay back their loans. We randomize the sequence of paybacks and returns.

In [72]:
BORROWER_ORDER=(`seq 1 $N_BORROWERS | sort -R`)
echo ${BORROWER_ORDER[@]}

3 2 5 4 1


In [73]:
for j in `seq 1 $N_BORROWERS`
do
  INTEREST_PERCENT=$((50 * RANDOM / 32767 - 10))
  RETURN_AMOUNT[$j]=$((${LOAN_AMOUNT[$j]} * (100 + INTEREST_PERCENT) / 100))
done
echo ${LOAN_AMOUNT[@]}
echo ${RETURN_AMOUNT[@]}

993462 239613 932569 222748 920703
1212023 333062 960546 307392 1178499


Pay back the loans.

In [74]:
for i in `seq 1 $N_BORROWERS`
do
  k1=$((k+1))
  k2=$((k+2))
  j=${BORROWER_ORDER[$((i-1))]}
  echo
  echo "--- Borrower deposit #$i ---"
  ROLE=${BORROWER_DEPOSIT_ROLE[$i]}
  ADDR=${BORROWER_ADDR[$j]}
  SKEY=${BORROWER_SKEY[$j]}
  AMOUNT=${RETURN_AMOUNT[$j]}
  DEADLINE=$((LOAN_REPAYMENT_DEADLINE - 1 * MINUTE))
  echo "Borrower $j requests to deposit $AMOUNT DjedUSD."
  marlowe-cli run prepare --choice-party Role=$ADMIN_ROLE            \
                          --choice-name "Borrower Deposit"           \
                          --choice-number $AMOUNT                    \
                          --invalid-before $NOW                      \
                          --invalid-hereafter $DEADLINE              \
                          --marlowe-file collective-loan-$k.marlowe  \
                          --out-file     collective-loan-$k1.marlowe
  echo "Administrator approves deposit and sends role token to borrower."
  TX_ADA=$(find_ada $ADMIN_ADDR)
  TX_NEW=$(
  marlowe-cli run execute --marlowe-in-file collective-loan-$k.marlowe          \
                          --tx-in-marlowe "$TX_OLD#1"                           \
                          --tx-in-collateral $TX_ADA                            \
                          --tx-in $TX_ADA                                       \
                          --tx-in "$(find_role_token $ADMIN_ADDR $ADMIN_ROLE)"  \
                          --tx-in "$(find_role_token $ADMIN_ADDR $ROLE)"        \
                          --tx-out "$(make_role_txout $ADMIN_ADDR $ADMIN_ROLE)" \
                          --tx-out "$(make_role_txout $ADDR $ROLE)"             \
                          --marlowe-out-file collective-loan-$k1.marlowe        \
                          --change-address $ADMIN_ADDR                          \
                          --required-signer $ADMIN_SKEY                         \
                          --out-file /dev/null                                  \
                          --submit 600                                          \
  | sed -e 's/^TxId "\(.*\)"$/\1/'                                              \
  )
  echo 'TxId "'$TX_NEW'"'
  TX_OLD=$TX_NEW
  echo "Borrower makes deposit using their newly-received role token."
  marlowe-cli run prepare --deposit-account Role=$ADMIN_ROLE         \
                          --deposit-party Role=$ROLE                 \
                          --deposit-amount $AMOUNT                   \
                          --deposit-token $DJED                      \
                          --invalid-before $NOW                      \
                          --invalid-hereafter $DEADLINE              \
                          --marlowe-file collective-loan-$k1.marlowe \
                          --out-file     collective-loan-$k2.marlowe 
  TX_ADA=$(find_ada $ADDR)
  CHANGE=$((BORROWER_DJED + LOAN_AMOUNT[$j] - AMOUNT))
  TX_NEW=$(
  marlowe-cli run execute --marlowe-in-file collective-loan-$k1.marlowe      \
                          --tx-in-marlowe "$TX_OLD#1"                        \
                          --tx-in-collateral $TX_ADA                         \
                          --tx-in $TX_ADA                                    \
                          --tx-in "$(find_token $ADDR $DJED)"                \
                          --tx-in "$(find_token_2 $ADDR $DJED)"              \
                          --tx-in "$(find_role_token $ADDR $ROLE)"           \
                          --tx-out "$(make_role_txout $ADDR $ROLE)"          \
                          --tx-out "$(make_token_txout $ADDR $DJED $CHANGE)" \
                          --marlowe-out-file collective-loan-$k2.marlowe     \
                          --change-address $ADDR                             \
                          --required-signer $SKEY                            \
                          --out-file /dev/null                               \
                          --submit 600                                       \
  | sed -e 's/^TxId "\(.*\)"$/\1/'                                           \
  )
  echo 'TxId "'$TX_NEW'"'
  TX_OLD=$TX_NEW
  k=$k2
done


--- Borrower deposit #1 ---
Borrower 3 requests to deposit 960546 DjedUSD.
Administrator approves deposit and sends role token to borrower.
TxId "3479bbbee20a28f4257d445224f92974f5f9b00ce2d6e2c9076dc9642dbcf0c7"
Borrower makes deposit using their newly-received role token.
TxId "a99a1e8fb552d2a388d2640f270a426704a86ccb9677559da5fba70f89103e33"

--- Borrower deposit #2 ---
Borrower 2 requests to deposit 333062 DjedUSD.
Administrator approves deposit and sends role token to borrower.
TxId "7908e66856590f6b0e8871bf9805992d5846e7306aca40eaf21805112df233eb"
Borrower makes deposit using their newly-received role token.
TxId "cc40e01f5f165cae65f547d5a5b403ac3d369c7833253e781c591547c141ca35"

--- Borrower deposit #3 ---
Borrower 5 requests to deposit 1178499 DjedUSD.
Administrator approves deposit and sends role token to borrower.
TxId "ddc7cd4b673864c451a300b1cb46df1cec6fd3d98e9063d594899b56ea620e76"
Borrower makes deposit using their newly-received role token.
TxId "5b15c1920b0dda11fbcc2c1b

Now the contract accounts show that the Djed balance has increased.

In [75]:
jq .state.accounts collective-loan-$k.marlowe | json2yaml

- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: A
    - currency_symbol: 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd
      token_name: Liquidity
  - 9331525
- - - role_token: A
    - currency_symbol: 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf
      token_name: DjedUSD
  - 6350902


The contract has an excess of assets over liabilities.

In [76]:
jq .state.boundValues collective-loan-$k.marlowe | json2yaml

- - Liability
  - 5668475
- - Equity
  - 6350902


View the UTxO at the script address.

In [77]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
4a265809b0f51def6d24f30d12e48b7e3d56766ffad88123e1302e1d51698813     1        2000000 lovelace + 9331525 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + 6350902 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344 + TxOutDatumHash ScriptDataInAlonzoEra "f955fd24188ad5972c1b814349978b08633e7d1a1347b0455f0745d32f421cfd"


There are no UTxOs at the payout address.

In [78]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ROLES_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


Defragment and then view the UTxOs at the borrowers' addresses to see that they have their role tokens and Djed.

In [79]:
for j in `seq 1 $N_BORROWERS`
do
  echo
  echo "Borrower $j = ${BORROWER_ADDR[$j]}"
  marlowe-cli util clean --required-signer ${BORROWER_SKEY[$j]} --change-address ${BORROWER_ADDR[$j]} --out-file /dev/null --submit 600
  cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address ${BORROWER_ADDR[$j]}
done


Borrower 1 = addr_test1qp7q4e9nlhq08mrugs8pm5uq045e425w72mxhxupq9qqgykr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqjmt3lh
TxId "0a36c87b854ab158ede79ef82ff5d911e7624503fed4ca2584f79d44f0b01860"
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
0a36c87b854ab158ede79ef82ff5d911e7624503fed4ca2584f79d44f0b01860     0        418580680 lovelace + TxOutDatumNone
0a36c87b854ab158ede79ef82ff5d911e7624503fed4ca2584f79d44f0b01860     1        2000000 lovelace + 281439 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344 + TxOutDatumNone
0a36c87b854ab158ede79ef82ff5d911e7624503fed4ca2584f79d44f0b01860     2        2000000 lovelace + 1 9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064e78.424435 + TxOutDatumNone
0a36c87b854ab158ede79ef82ff5d911e7624503fed4ca2584f79d44f0b01860     3        2000000 lovelace + 1 9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d

### 7. The lenders redeem their liquidity tokens

Lenders redeem in random order.

In [80]:
INVESTMENT_ORDER=(`seq 1 $N_LENDERS | sort -R`)
echo ${INVESTMENT_ORDER[@]}

2 1 3


Redeem the proceeds of the investments.

In [81]:
for i in `seq 1 $N_LENDERS`
do
  k1=$((k+1))
  k2=$((k+2))
  j=${INVESTMENT_ORDER[$((i-1))]}
  echo
  echo "--- Lender redemption #$i ---"
  ROLE=${LENDER_PAYMENT_ROLE[$i]}
  ADDR=${LENDER_ADDR[$j]}
  SKEY=${LENDER_SKEY[$j]}
  AMOUNT=${INVESTMENT_AMOUNT[$j]}
  DEADLINE=$((INVESTMENT_REPAYMENT_DEADLINE - 1 * MINUTE))
  echo "Lender $j requests to deposit $AMOUNT liquidity tokens."
  marlowe-cli run prepare --choice-party Role=$ADMIN_ROLE            \
                          --choice-name "Lender Withdrawal"          \
                          --choice-number $AMOUNT                    \
                          --invalid-before $NOW                      \
                          --invalid-hereafter $DEADLINE              \
                          --marlowe-file collective-loan-$k.marlowe  \
                          --out-file     collective-loan-$k1.marlowe
  echo "Administrator approves deposit and sends role token to lender."
  TX_ADA=$(find_ada $ADMIN_ADDR)
  TX_NEW=$(
  marlowe-cli run execute --marlowe-in-file collective-loan-$k.marlowe          \
                          --tx-in-marlowe "$TX_OLD#1"                           \
                          --tx-in-collateral $TX_ADA                            \
                          --tx-in $TX_ADA                                       \
                          --tx-in "$(find_role_token $ADMIN_ADDR $ADMIN_ROLE)"  \
                          --tx-in "$(find_role_token $ADMIN_ADDR $ROLE)"        \
                          --tx-out "$(make_role_txout $ADMIN_ADDR $ADMIN_ROLE)" \
                          --tx-out "$(make_role_txout $ADDR $ROLE)"             \
                          --marlowe-out-file collective-loan-$k1.marlowe        \
                          --change-address $ADMIN_ADDR                          \
                          --required-signer $ADMIN_SKEY                         \
                          --out-file /dev/null                                  \
                          --submit 600                                          \
  | sed -e 's/^TxId "\(.*\)"$/\1/'                                              \
  )
  echo 'TxId "'$TX_NEW'"'
  TX_OLD=$TX_NEW
  echo "Lender deposits liquidity tokens using their newly-received role token."
  marlowe-cli run prepare --deposit-account Role=$ADMIN_ROLE         \
                          --deposit-party Role=$ROLE                 \
                          --deposit-amount $AMOUNT                   \
                          --deposit-token $LIQUIDITY                 \
                          --invalid-before $NOW                      \
                          --invalid-hereafter $DEADLINE              \
                          --marlowe-file collective-loan-$k1.marlowe \
                          --out-file     collective-loan-$k2.marlowe 
  TX_ADA=$(find_ada $ADDR)
  TX_NEW=$(
  marlowe-cli run execute --marlowe-in-file collective-loan-$k1.marlowe  \
                          --tx-in-marlowe "$TX_OLD#1"                    \
                          --tx-in-collateral $TX_ADA                     \
                          --tx-in $TX_ADA                                \
                          --tx-in "$(find_token $ADDR $LIQUIDITY)"       \
                          --tx-in "$(find_role_token $ADDR $ROLE)"       \
                          --tx-out "$(make_role_txout $ADDR $ROLE)"      \
                          --marlowe-out-file collective-loan-$k2.marlowe \
                          --change-address $ADDR                         \
                          --required-signer $SKEY                        \
                          --out-file /dev/null                           \
                          --submit 600                                   \
  | sed -e 's/^TxId "\(.*\)"$/\1/'                                       \
  )
  echo 'TxId "'$TX_NEW'"'
  TX_OLD=$TX_NEW
  echo "Lender withdraws their funds from the payout address."
  TX_ADA=$(find_ada $ADDR)
  marlowe-cli run withdraw --marlowe-file collective-loan-$k2.marlowe \
                           --role-name $ROLE                          \
                           --tx-in-collateral $TX_ADA                 \
                           --tx-in $TX_ADA                            \
                           --tx-in $(find_role_token $ADDR $ROLE)     \
                           --tx-out "$(make_role_txout $ADDR $ROLE)"  \
                           --change-address $ADDR                     \
                           --required-signer $SKEY                    \
                           --out-file /dev/null                       \
                           --submit 600
  k=$k2
done


--- Lender redemption #1 ---
Lender 2 requests to deposit 2056260 liquidity tokens.
Administrator approves deposit and sends role token to lender.
TxId "218d54a7e95379cee16025c5f25e775f2386125b92bdf1657cfff93a1d02b711"
Lender deposits liquidity tokens using their newly-received role token.
Payment 1
  Acccount: "A"
  Payee: Party "LP1"
  Ada: 0.000000
  76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf."DjedUSD": 2303813
TxId "99b6529a5a7f3569c5384c32f04445008c8dfb576d3fa06a043df2f76299f06e"
Lender withdraws their funds from the payout address.
TxId "db74a9327e4d2ead679074e878a320151e9287a373cdf5f2db75c31b2a53d2e3"

--- Lender redemption #2 ---
Lender 1 requests to deposit 671391 liquidity tokens.
Administrator approves deposit and sends role token to lender.
TxId "dfc739d481a9115abb3466b6b0d3d003883a88648883ad969db460ec29239b26"
Lender deposits liquidity tokens using their newly-received role token.
Payment 1
  Acccount: "A"
  Payee: Party "LP2"
  Ada: 0.000000
  76ea6a9aad89b

Now the contract accounts show that the contract only retains the liquidity tokens.

In [82]:
jq .state.accounts collective-loan-$k.marlowe | json2yaml

- - - role_token: A
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: A
    - currency_symbol: 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd
      token_name: Liquidity
  - 15000000


View the UTxO at the script address.

In [83]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
01e8864a819588220d7b2693fb9ff792883c74f292e33eae327156851e16e0b2     1        2000000 lovelace + 15000000 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + TxOutDatumHash ScriptDataInAlonzoEra "d219dfd716b05b0417b48f6dcc255578d27e1c89d9858dbb85a47f46a2fd90ea"


There are no UTxOs at the payout address.

In [84]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ROLES_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


Defragment and then view the UTxOs at the lenders' addresses to see that they have their role tokens and liquidity tokens.

In [85]:
for j in `seq 1 $N_LENDERS`
do
  echo
  echo "Lender $j = ${LENDER_ADDR[$j]}"
  marlowe-cli util clean --required-signer ${LENDER_SKEY[$j]} --change-address ${LENDER_ADDR[$j]} --out-file /dev/null --submit 600
  cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address ${LENDER_ADDR[$j]}
done


Lender 1 = addr_test1qzenwj7elwatk3mmc368mwnv067fqnuk3x7u4nqw5dezg8wr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jqf6k36p
TxId "da9c00fca523f3ecd3cf7a152ae90f3a04500937a57b0e673f5b129eaa191302"
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
da9c00fca523f3ecd3cf7a152ae90f3a04500937a57b0e673f5b129eaa191302     0        362939613 lovelace + TxOutDatumNone
da9c00fca523f3ecd3cf7a152ae90f3a04500937a57b0e673f5b129eaa191302     1        2000000 lovelace + 5080829 76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf.446a6564555344 + TxOutDatumNone
da9c00fca523f3ecd3cf7a152ae90f3a04500937a57b0e673f5b129eaa191302     2        2000000 lovelace + 1 9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064e78.4c4431 + TxOutDatumNone
da9c00fca523f3ecd3cf7a152ae90f3a04500937a57b0e673f5b129eaa191302     3        2000000 lovelace + 1 9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d6

### 8. Close the contract

The administrator receives back their initial Ada and liquidity tokens.

In [86]:
marlowe-cli run prepare --notify                                                \
                        --invalid-before $NOW                                   \
                        --invalid-hereafter $((ULTIMATE_DEADLINE - 1 * MINUTE)) \
                        --marlowe-file collective-loan-$k.marlowe               \
                        --out-file     collective-loan-$((1+k)).marlowe

Payment 1
  Acccount: "A"
  Payee: Party "A"
  Ada: 2.000000
Payment 2
  Acccount: "A"
  Payee: Party "A"
  Ada: 0.000000
  5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd."Liquidity": 15000000


Submit the transactions.

In [87]:
TX_ADA=$(find_ada $ADMIN_ADDR)
marlowe-cli run execute --marlowe-in-file collective-loan-$k.marlowe          \
                        --tx-in-marlowe "$TX_OLD#1"                           \
                        --tx-in-collateral $TX_ADA                            \
                        --tx-in $TX_ADA                                       \
                        --tx-in "$(find_role_token $ADMIN_ADDR $ADMIN_ROLE)"  \
                        --tx-out "$(make_role_txout $ADMIN_ADDR $ADMIN_ROLE)" \
                        --marlowe-out-file collective-loan-$((k+1)).marlowe   \
                        --change-address $ADMIN_ADDR                          \
                        --required-signer $ADMIN_SKEY                         \
                        --out-file /dev/null                                  \
                        --submit 600      

TxId "50ccb80d408689a8060b17fe9e81b15f2e3c4e77527304051140237988e2ba40"


In [88]:
TX_ADA=$(find_ada $ADMIN_ADDR)
marlowe-cli run withdraw --marlowe-file collective-loan-$((k+1)).marlowe       \
                         --role-name $ADMIN_ROLE                               \
                         --tx-in-collateral $TX_ADA                            \
                         --tx-in $TX_ADA                                       \
                         --tx-in $(find_role_token $ADMIN_ADDR $ADMIN_ROLE)    \
                         --tx-out "$(make_role_txout $ADMIN_ADDR $ADMIN_ROLE)" \
                         --change-address $ADMIN_ADDR                          \
                         --required-signer $ADMIN_SKEY                         \
                         --out-file /dev/null                                  \
                         --submit 600

TxId "c9ba7397c70f480f0ab6578e81ced30f659886e9e5a3fa425b12a405d6e5e7a8"


There is no UTxO at the script address.

In [89]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $SCRIPT_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


There are no UTxOs at the payout address.

In [90]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ROLES_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


Defragment and then view the UTxOs at the administrator's address.

In [91]:
marlowe-cli util clean --required-signer $ADMIN_SKEY --change-address $ADMIN_ADDR --out-file /dev/null --submit 600
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ADMIN_ADDR

TxId "6e939cfbde7b02458da795102f7cdfa0be13f4d3c444559204e70c72f89f278b"
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
6e939cfbde7b02458da795102f7cdfa0be13f4d3c444559204e70c72f89f278b     0        1957201789 lovelace + TxOutDatumNone
6e939cfbde7b02458da795102f7cdfa0be13f4d3c444559204e70c72f89f278b     1        2000000 lovelace + 15000000 5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd.4c6971756964697479 + TxOutDatumNone
6e939cfbde7b02458da795102f7cdfa0be13f4d3c444559204e70c72f89f278b     2        2000000 lovelace + 1 9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064e78.41 + TxOutDatumNone


## Clean Up

Use Daedalus to send all of the lender and borrower tokens to the administrator.

In [92]:
echo $ADMIN_ADDR
false

addr_test1vrssw4edcts00kk6lp7p5n64666m23tpprqaarmdwkaq69gfvqnpz


: 1

Burn the Djed.

In [93]:
marlowe-cli util mint --required-signer $ADMIN_SKEY \
                      --change-address $ADMIN_ADDR  \
                      --expires 1000000011          \
                      --count -$TOTAL_DJED          \
                      --out-file /dev/null          \
                      --submit 600                  \
                      $DJED_NAME

PolicyID "76ea6a9aad89b09448e7af765f5a3e89c574989d67d6371a80e58baf"


Burn the liquidity tokens.

In [94]:
marlowe-cli util mint --required-signer $ADMIN_SKEY \
                      --change-address $ADMIN_ADDR  \
                      --expires 1000000012          \
                      --count -$POOL_SIZE           \
                      --out-file /dev/null          \
                      --submit 600                  \
                      $LIQUIDITY_NAME

PolicyID "5f10ff15e04456ec25f1a2d64e5584087e584613411e8e9051b70fbd"


Burn the role tokens.

In [95]:
marlowe-cli util mint --required-signer $ADMIN_SKEY \
                      --change-address $ADMIN_ADDR  \
                      --expires 1000000013          \
                      --count -1                    \
                      --out-file /dev/null          \
                      --submit 600                  \
                      $ADMIN_ROLE                   \
                      ${LENDER_DEPOSIT_ROLE[@]}     \
                      ${LENDER_PAYMENT_ROLE[@]}     \
                      ${BORROWER_DEPOSIT_ROLE[@]}   \
                      ${BORROWER_PAYMENT_ROLE[@]}

PolicyID "9cb30bdd840cc8b31806c20eae6ed14614d32f10379547d605064e78"


Check that all of the burning was successful.

In [96]:
cardano-cli query utxo --testnet-magic $CARDANO_TESTNET_MAGIC --address $ADMIN_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
8a1223d920eb6674ab94db7434218e4e04874affaffcfb94dff6694c660a3e44     0        1962860880 lovelace + TxOutDatumNone
